In [1]:
import warnings
import time
import sys
import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

warnings.simplefilter(action='ignore', category=FutureWarning)

import winsound
frequency = 400  # Hz
duration = 700    # milliseconds

pd.set_option('display.max_columns', None)

In [2]:
def preprocessing(df):
    for col in ['authorized_flag', 'category_1']:
        df[col] = df[col].map({'Y':1, 'N':0})
    for col in ['category_3']:
        df[col] = df[col].map({'A':1, 'B':2,'C':3})
    return df

def read_data(input_file):
    df = pd.read_csv(input_file)
    df['first_active_month'] = pd.to_datetime(df['first_active_month'])
    df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days
    return df

In [3]:
new_merchant_transactions = pd.read_csv('../Kaggle-data/new_merchant_transactions.csv', parse_dates=['purchase_date'])
historical_transactions = pd.read_csv('../Kaggle-data/historical_transactions.csv', parse_dates=['purchase_date'])
merchants = pd.read_csv('../Kaggle-data/merchants.csv')
train = pd.read_csv('../Kaggle-data/train.csv', parse_dates=['first_active_month'])
test = pd.read_csv('../Kaggle-data/test.csv', parse_dates=['first_active_month'])
winsound.Beep(frequency, duration)

In [4]:
# 問題が起きればここから回す
new_transactions = new_merchant_transactions.copy()
hist_transactions = historical_transactions.copy()
merchants_temp = merchants.copy()
train_temp = train.copy()
test_temp = test.copy()

In [5]:
new = preprocessing(new_transactions)
hist = preprocessing(hist_transactions)

In [6]:
# # installmentsが999のものを削除
# new = new[new['installments'] != 999]
# hist = hist[hist['installments'] != 999]

In [7]:
merchants_temp.isnull().sum()

merchant_id                        0
merchant_group_id                  0
merchant_category_id               0
subsector_id                       0
numerical_1                        0
numerical_2                        0
category_1                         0
most_recent_sales_range            0
most_recent_purchases_range        0
avg_sales_lag3                    13
avg_purchases_lag3                 0
active_months_lag3                 0
avg_sales_lag6                    13
avg_purchases_lag6                 0
active_months_lag6                 0
avg_sales_lag12                   13
avg_purchases_lag12                0
active_months_lag12                0
category_4                         0
city_id                            0
state_id                           0
category_2                     11887
dtype: int64

In [8]:
hist.head(5)

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id
0,1,C_ID_4e6213e9bc,88,0,0,1.0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37
1,1,C_ID_4e6213e9bc,88,0,0,1.0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16
2,1,C_ID_4e6213e9bc,88,0,0,1.0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37
3,1,C_ID_4e6213e9bc,88,0,0,1.0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34
4,1,C_ID_4e6213e9bc,88,0,0,1.0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37


In [9]:
# target < -30は除去
# train_temp = train.query('target > -30')

In [10]:
for df in [test, train]:
    # https://www.kaggle.com/code/mfjwr1/simple-lightgbm-without-blending
    df['elapsed_time'] = (pd.Timestamp('2018-02-01') - df['first_active_month']).dt.days # 2018-02-01がmax
    df['quarter'] = df['first_active_month'].dt.quarter
    df['days_feature1'] = df['elapsed_time'] * df['feature_1']
    df['days_feature2'] = df['elapsed_time'] * df['feature_2']
    df['days_feature3'] = df['elapsed_time'] * df['feature_3']
    df['days_feature1_ratio'] = df['feature_1'] / df['elapsed_time']
    df['days_feature2_ratio'] = df['feature_2'] / df['elapsed_time']
    df['days_feature3_ratio'] = df['feature_3'] / df['elapsed_time']
    df['feature_sum'] = df['feature_1'] + df['feature_2'] + df['feature_3']
    df['feature_product'] = df['feature_1'] * df['feature_2'] * df['feature_3'] # Kawai added
    df['feature_mean'] = df['feature_sum'] / 3
    df['feature_max'] = df[['feature_1', 'feature_2', 'feature_3']].max(axis=1)
    df['feature_min'] = df[['feature_1', 'feature_2', 'feature_3']].min(axis=1)
    df['feature_var'] = df[['feature_1', 'feature_2', 'feature_3']].std(axis=1)

In [11]:
for df in [test, train]:
    df['first_active_month']  = pd.to_datetime(df['first_active_month'])

- target_rawを予測するモデルにする

In [12]:
train['target_raw'] = 2 ** train['target']
train.drop('target', axis=1, inplace=True)

In [13]:
for df in [hist, new]:
    df['purchase_amount_new'] = np.round(df['purchase_amount'] / 0.00150265118 + 497.06)

In [14]:
for df in [hist, new]:
    df['category_2'].fillna(1, inplace=True) # 1が最頻値
    df['category_3'].fillna(1, inplace=True) # 1が最頻値
    df['merchant_id'].fillna('M_ID_00a6ca8a8a',inplace=True) # 最頻値

    # https://www.kaggle.com/code/mfjwr1/simple-lightgbm-without-blending
    df['installments'].replace(-1, np.nan, inplace=True)
    df['installments'].replace(999, np.nan, inplace=True)
    df['purchase_date'] = pd.to_datetime(df['purchase_date'])
    df['month'] = df['purchase_date'].dt.month
    df['day'] = df['purchase_date'].dt.day
    df['hour'] = df['purchase_date'].dt.hour
    #df['weekofyear'] = df['purchase_date'].dt.weekofyear
    df['weekday'] = df['purchase_date'].dt.weekday
    df['weekend'] = (df['purchase_date'].dt.weekday >=5).astype(int)
    # additional features
    df['price'] = df['purchase_amount_new'] / df['installments']
    # additional features
    df['month_diff'] = ((datetime.datetime.today() - df['purchase_date']).dt.days)//30
    df['month_diff'] += df['month_lag']
    df['duration'] = df['purchase_amount_new']*df['month_diff']
    df['amount_month_ratio'] = df['purchase_amount_new'] / df['month_diff']

In [15]:
hist['price'] = hist['price'].replace([np.inf, -np.inf], np.nan)
new['price'] = new['price'].replace([np.inf, -np.inf], np.nan)

In [16]:
hist

,authorized_flag,card_id,city_id,category_1,installments,category_3,merchant_category_id,merchant_id,month_lag,purchase_amount,purchase_date,category_2,state_id,subsector_id,purchase_amount_new,month,day,hour,weekday,weekend,price,month_diff,duration,amount_month_ratio
0,1,C_ID_4e6213e9bc,88,0,0.0,1.0,80,M_ID_e020e9b302,-8,-0.703331,2017-06-25 15:33:07,1.0,16,37,29.0,6,25,15,6,1,NaN,77,2233.0,0.376623
1,1,C_ID_4e6213e9bc,88,0,0.0,1.0,367,M_ID_86ec983688,-7,-0.733128,2017-07-15 12:10:45,1.0,16,16,9.0,7,15,12,5,1,NaN,77,693.0,0.116883
2,1,C_ID_4e6213e9bc,88,0,0.0,1.0,80,M_ID_979ed661fc,-6,-0.720386,2017-08-09 22:04:29,1.0,16,37,18.0,8,9,22,2,0,NaN,77,1386.0,0.233766
3,1,C_ID_4e6213e9bc,88,0,0.0,1.0,560,M_ID_e6d5ae8ea6,-5,-0.735352,2017-09-02 10:06:26,1.0,16,34,8.0,9,2,10,5,1,NaN,77,616.0,0.103896
4,1,C_ID_4e6213e9bc,88,0,0.0,1.0,80,M_ID_e020e9b302,-11,-0.722865,2017-03-10 01:14:19,1.0,16,37,16.0,3,10,1,4,0,NaN,77,1232.0,0.207792
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29112356,1,C_ID_2863d2fa95,-1,1,1.0,2.0,360,M_ID_edd92b6720,-1,-0.632706,2017-01-20 08:52:04,1.0,-1,34,76.0,1,20,8,4,0,76.0,89,6764.0,0.853933
29112357,1,C_ID_2863d2fa95,-1,1,1.0,2.0,360,M_ID_edd92b6720,0,-0.632706,2017-02-20 04:40:50,1.0,-1,34,76.0,2,20,4,0,0,76.0,89,6764.0,0.853933
29112358,1,C_ID_5c240d6e3c,3,0,0.0,1.0,278,M_ID_9cdcfe8673,0,-0.657740,2017-12-26 18:37:51,1.0,16,37,59.0,12,26,18,1,0,NaN,79,4661.0,0.746835
29112359,1,C_ID_5c240d6e3c,331,0,0.0,1.0,514,M_ID_1a75f94f92,-1,0.770620,2017-11-24 14:18:15,1.0,16,9,1010.0,11,24,14,4,0,NaN,79,79790.0,12.784810


In [17]:
# 承認フラグ　Yes,Noの数と比
# histのみ（newはYesのみのため）
authorized_flag = hist[['card_id','authorized_flag']].groupby(['card_id','authorized_flag']).agg(count=('card_id','count')).reset_index()

# カードIDごとの合計を計算
total_counts = authorized_flag.groupby('card_id')['count'].sum().reset_index()
total_counts = total_counts.rename(columns={'count': 'total_count'})

# 元のデータと合計を結合
authorized_flag = authorized_flag.merge(total_counts, on='card_id')

# 割合を計算
authorized_flag['ratio'] = authorized_flag['count'] / authorized_flag['total_count']
#hist[['card_id','authorized_flag']].groupby(['card_id']).agg(count=('authorized_flag','mean')).reset_index()

# 結果を整形
authorized_ratio = authorized_flag.pivot(index='card_id', columns='authorized_flag', values=['ratio','count']).reset_index().rename(columns={0: 'No', 1: 'Yes'})
authorized_ratio.columns = ['_'.join(col).strip() for col in authorized_ratio.columns.values]
authorized_ratio.rename(columns={'card_id_': 'card_id'}, inplace=True)

In [18]:
authorized_ratio

,card_id,ratio_No,ratio_Yes,count_No,count_Yes
0,C_ID_00007093c1,0.234899,0.765101,35.0,114.0
1,C_ID_0001238066,0.024390,0.975610,3.0,120.0
2,C_ID_0001506ef0,0.060606,0.939394,4.0,62.0
3,C_ID_0001793786,0.125000,0.875000,27.0,189.0
4,C_ID_000183fdda,0.048611,0.951389,7.0,137.0
...,...,...,...,...,...
325535,C_ID_ffff1d9928,0.250000,0.750000,3.0,9.0
325536,C_ID_ffff579d3a,0.131579,0.868421,15.0,99.0
325537,C_ID_ffff756266,0.416667,0.583333,10.0,14.0
325538,C_ID_ffff828181,0.126316,0.873684,24.0,166.0


In [19]:
total = pd.concat([hist,new],axis=0).reset_index(drop=True)

In [20]:
# total

In [21]:
# hist_id_u = hist.groupby('card_id', as_index=False)\
#     .agg(
#         frequency = ('card_id', 'count'),
#         amount_total = ('purchase_amount_new', 'sum'),
#         amount_mean = ('purchase_amount_new', 'mean'),
#         amount_max = ('purchase_amount_new', 'max'),
#         amount_min = ('purchase_amount_new', 'min'),
#         amount_median = ('purchase_amount_new', 'median'),
#         merchant_category_nu = ('merchant_category_id', 'nunique'),
#         merchant_category_mode = ('merchant_category_id', lambda x: x.mode()[0]),
#         city_nu = ('city_id', 'nunique'),
#         city_mode = ('city_id', lambda x: x.mode()[0]),
#         state_nu = ('state_id', 'nunique'),
#         state_mode = ('state_id', lambda x: x.mode()[0]),
#         date_min  = ('purchase_date', 'min'),
#         date_max  = ('purchase_date', 'max'),
#         installments_mean = ('installments', 'mean'),
#         installments_max = ('installments', 'max'),
#         installments_min = ('installments', 'min'),
#         installments_median = ('installments', 'median'),
#         installments_sum = ('installments', 'sum'),
#         month_lag_mean = ('month_lag', 'mean'),
#         month_lag_max = ('month_lag', 'max'),
#         month_lag_min = ('month_lag', 'min'),
#         month_lag_median = ('month_lag', 'median'),
#         category_1_mod = ('category_1', lambda x: x.mode()[0]),
#         category_2_mod = ('category_2', lambda x: x.mode()[0]),
#         category_3_mod = ('category_3', lambda x: x.mode()[0]),
#         # additional
#         weekend_mean = ('weekend', 'mean'),
#         weekday_mean = ('weekday', 'mean'),
#         day_nu = ('day', 'nunique'),
#         day_min = ('day', 'min'),
#         day_max = ('day', 'max'),
#         price_sum = ('price', 'sum'),
#         price_min = ('price', 'min'),
#         price_max = ('price', 'max'),
#         price_mean = ('price', 'mean'),
#         duration_min = ('duration', 'min'),
#         duration_max = ('duration', 'max'),
#         duration_mean = ('duration', 'mean'),
#         amount_month_ratio_min = ('amount_month_ratio', 'min'),
#         amount_month_ratio_max = ('amount_month_ratio', 'max'),
#         amount_month_ratio_mean = ('amount_month_ratio', 'mean')
#     )

In [22]:
# hist_id_u = hist_id_u.merge(authorized_ratio, on='card_id')

In [23]:
# hist_id_u['duration'] = (hist_id_u['date_max'] - hist_id_u['date_min']).dt.days
# hist_id_u.date_max = (hist_id_u.date_max - hist_id_u.date_min.min()).dt.days
# hist_id_u.date_min = (hist_id_u.date_min - hist_id_u.date_min.min()).dt.days

# hist_id_u.merchant_category_mode = pd.to_numeric(hist_id_u.merchant_category_mode, errors='coerce')

# hist_id_u.columns = ['hist_' + col if col != 'card_id' else col for col in hist_id_u.columns]

In [24]:
# new_id_u = new.groupby('card_id', as_index=False)\
#     .agg(
#         frequency = ('card_id', 'count'),
#         amount_total = ('purchase_amount_new', 'sum'),
#         amount_mean = ('purchase_amount_new', 'mean'),
#         amount_max = ('purchase_amount_new', 'max'),
#         amount_min = ('purchase_amount_new', 'min'),
#         amount_median = ('purchase_amount', 'median'),
#         merchant_category_nu = ('merchant_category_id', 'nunique'),
#         merchant_category_mode = ('merchant_category_id', lambda x: x.mode()[0]),
#         city_nu = ('city_id', 'nunique'),
#         city_mode = ('city_id', lambda x: x.mode()[0]),
#         state_nu = ('state_id', 'nunique'),
#         state_mode = ('state_id', lambda x: x.mode()[0]),
#         date_min  = ('purchase_date', 'min'),
#         date_max  = ('purchase_date', 'max'),
#         installments_mean = ('installments', 'mean'),
#         installments_max = ('installments', 'max'),
#         installments_min = ('installments', 'min'),
#         installments_median = ('installments', 'median'),
#         installments_sum = ('installments', 'sum'),
#         month_lag_mean = ('month_lag', 'mean'),
#         month_lag_max = ('month_lag', 'max'),
#         month_lag_min = ('month_lag', 'min'),
#         month_lag_median = ('month_lag', 'median'),
#         category_1_mod = ('category_1', lambda x: x.mode()[0]),
#         category_2_mod = ('category_2', lambda x: x.mode()[0]),
#         category_3_mod = ('category_3', lambda x: x.mode()[0]),
#         # additional
#         weekend_mean = ('weekend', 'mean'),
#         weekday_mean = ('weekday', 'mean'),
#         day_nu = ('day', 'nunique'),
#         day_min = ('day', 'min'),
#         day_max = ('day', 'max'),
#         price_sum = ('price', 'sum'),
#         price_min = ('price', 'min'),
#         price_max = ('price', 'max'),
#         price_mean = ('price', 'mean'),
#         duration_min = ('duration', 'min'),
#         duration_max = ('duration', 'max'),
#         duration_mean = ('duration', 'mean'),
#         amount_month_ratio_min = ('amount_month_ratio', 'min'),
#         amount_month_ratio_max = ('amount_month_ratio', 'max'),
#         amount_month_ratio_mean = ('amount_month_ratio', 'mean')
#     )

In [25]:
# new_id_u['duration'] = (new_id_u['date_max'] - new_id_u['date_min']).dt.days
# new_id_u.date_max = (new_id_u.date_max - new_id_u.date_min.min()).dt.days
# new_id_u.date_min = (new_id_u.date_min - new_id_u.date_min.min()).dt.days

# new_id_u.merchant_category_mode = pd.to_numeric(new_id_u.merchant_category_mode, errors='coerce')

# new_id_u.columns = ['new_' + col if col != 'card_id' else col for col in new_id_u.columns]

In [26]:
total_id_u = total.groupby('card_id',as_index=False)\
    .agg(
        frequency = ('card_id','count'),
        amount_total = ('purchase_amount_new', 'sum'),
        amount_mean = ('purchase_amount_new', 'mean'),
        amount_max = ('purchase_amount_new', 'max'),
        amount_min = ('purchase_amount_new', 'min'),
        amount_median = ('purchase_amount_new', 'median'),
        merchant_category_nu = ('merchant_category_id', 'nunique'),
        merchant_category_mode = ('merchant_category_id', lambda x: x.mode()[0]),
        city_nu = ('city_id', 'nunique'),
        city_mode = ('city_id', lambda x: x.mode()[0]),
        state_nu = ('state_id', 'nunique'),
        state_mode = ('state_id', lambda x: x.mode()[0]),
        date_min  = ('purchase_date', 'min'),
        date_max  = ('purchase_date', 'max'),
        installments_mean = ('installments', 'mean'),
        installments_max = ('installments', 'max'),
        installments_min = ('installments', 'min'),
        installments_median = ('installments', 'median'),
        installments_sum = ('installments', 'sum'),
        month_lag_mean = ('month_lag', 'mean'),
        month_lag_max = ('month_lag', 'max'),
        month_lag_min = ('month_lag', 'min'),
        month_lag_median = ('month_lag', 'median'),
        category_1_mod = ('category_1', lambda x: x.mode()[0]),
        category_2_mod = ('category_2', lambda x: x.mode()[0]),
        category_3_mod = ('category_3', lambda x: x.mode()[0]),
        # additional
        weekend_mean = ('weekend', 'mean'),
        weekday_mean = ('weekday', 'mean'),
        day_nu = ('day', 'nunique'),
        day_min = ('day', 'min'),
        day_max = ('day', 'max'),
        price_sum = ('price', 'sum'),
        price_min = ('price', 'min'),
        price_max = ('price', 'max'),
        price_mean = ('price', 'mean'),
        duration_min = ('duration', 'min'),
        duration_max = ('duration', 'max'),
        duration_mean = ('duration', 'mean'),
        amount_month_ratio_min = ('amount_month_ratio', 'min'),
        amount_month_ratio_max = ('amount_month_ratio', 'max'),
        amount_month_ratio_mean = ('amount_month_ratio', 'mean')
    )

In [27]:
total_id_u['duration'] = (total_id_u['date_max'] - total_id_u['date_min']).dt.days
total_id_u.date_max = (total_id_u.date_max - total_id_u.date_min.min()).dt.days
total_id_u.date_min = (total_id_u.date_min - total_id_u.date_min.min()).dt.days

total_id_u.merchant_category_mode = pd.to_numeric(total_id_u.merchant_category_mode, errors='coerce')

total_id_u.columns = ['total_' + col if col != 'card_id' else col for col in total_id_u.columns]

In [28]:
# total_id_u

In [29]:
total_id_u['total_amount_change_ratio'] = (total_id_u['total_amount_max'] - total_id_u['total_amount_mean']) / total_id_u['total_amount_mean']


In [30]:
# train = train.merge(hist_id_u, on='card_id', how='left')
# test = test.merge(hist_id_u, on='card_id', how='left')

# train = train.merge(new_id_u, on='card_id', how='left')
# test = test.merge(new_id_u, on='card_id', how='left')

train = train.merge(total_id_u, on='card_id', how='left')
test = test.merge(total_id_u, on='card_id', how='left')

In [31]:
train

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,quarter,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_product,feature_mean,feature_max,feature_min,feature_var,target_raw,total_frequency,total_amount_total,total_amount_mean,total_amount_max,total_amount_min,total_amount_median,total_merchant_category_nu,total_merchant_category_mode,total_city_nu,total_city_mode,total_state_nu,total_state_mode,total_date_min,total_date_max,total_installments_mean,total_installments_max,total_installments_min,total_installments_median,total_installments_sum,total_month_lag_mean,total_month_lag_max,total_month_lag_min,total_month_lag_median,total_category_1_mod,total_category_2_mod,total_category_3_mod,total_weekend_mean,total_weekday_mean,total_day_nu,total_day_min,total_day_max,total_price_sum,total_price_min,total_price_max,total_price_mean,total_duration_min,total_duration_max,total_duration_mean,total_amount_month_ratio_min,total_amount_month_ratio_max,total_amount_month_ratio_mean,total_duration,total_amount_change_ratio
0,2017-06-01,C_ID_92a2005557,5,2,1,245,2,1225,490,245,0.020408,0.008163,0.004082,8,10,2.666667,5,1,2.081666,0.566331,283,21403.0,75.628975,2000.0,5.0,34.0,46,560,9,69,3,9,177,483,0.014134,1.0,0.0,0.0,4.0,-3.473498,2,-8,-3.0,0,1.0,1.0,0.339223,3.204947,31,1,31,202.000000,6.0,87.0,50.500000,385.0,154000.0,5818.448763,0.064935,25.974026,0.983046,305,25.444891
1,2017-01-01,C_ID_3d0044924f,4,1,0,396,1,1584,396,0,0.010101,0.002525,0.000000,5,0,1.666667,4,0,2.081666,1.313042,356,34299.0,96.345506,3578.0,3.0,25.0,58,307,9,69,3,9,5,453,1.556497,10.0,1.0,1.0,551.0,-4.921348,2,-12,-5.0,0,1.0,2.0,0.370787,3.331461,31,1,31,16083.738095,3.0,1789.0,45.434288,234.0,279084.0,7509.575843,0.038462,45.871795,1.236099,447,36.137176
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549,3,1098,1098,0,0.003643,0.003643,0.000000,4,0,1.333333,2,0,1.154701,1.611111,44,1996.0,45.363636,400.0,11.0,31.5,9,705,5,143,2,5,10,482,0.000000,0.0,0.0,0.0,0.0,-8.363636,2,-13,-10.0,0,5.0,1.0,0.272727,3.340909,19,2,30,0.000000,NaN,NaN,NaN,847.0,30800.0,3489.318182,0.142857,5.194805,0.589767,472,7.817635
3,2017-09-01,C_ID_186d6a6901,4,3,0,153,3,612,459,0,0.026144,0.019608,0.000000,7,0,2.333333,4,0,2.081666,1.103813,84,5724.0,68.142857,1459.0,4.0,26.5,28,278,7,17,5,22,268,472,1.135802,3.0,1.0,1.0,92.0,-2.452381,2,-5,-3.5,0,4.0,2.0,0.166667,2.833333,26,2,31,4310.166667,4.0,1459.0,53.211934,308.0,112343.0,5244.071429,0.051948,18.948052,0.885473,203,20.410901
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92,4,92,276,0,0.010870,0.032609,0.000000,4,0,1.333333,3,0,1.527525,0.895181,169,38344.0,226.887574,5284.0,0.0,49.0,37,278,7,17,7,22,314,482,1.297619,12.0,1.0,1.0,218.0,-0.680473,2,-3,-1.0,0,4.0,2.0,0.319527,3.248521,31,1,31,16933.333333,0.0,797.0,100.793651,0.0,401584.0,17340.721893,0.000000,69.526316,2.968742,167,22.289067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,3,2,1,153,3,459,306,153,0.019608,0.013072,0.006536,6,6,2.000000,3,1,1.000000,0.149600,47,1189.0,25.297872,116.0,3.0,12.0,11,705,5,25,2,7,249,434,0.021277,1.0,0.0,0.0,1.0,-2.000000,1,-5,-2.0,0,3.0,1.0,0.361702,3.659574,22,1,29,45.000000,45.0,45.0,45.000000,228.0,8932.0,1947.531915,0.038961,1.506494,0.328613,184,3.585366
201913,2015-10-01,C_ID_1314773c0b,3,1,1,854,4,2562,854,854,0.003513,0.001171,0.001171,5,3,1.666667,3,1,1.154701,1.242217,48,1111.0,23.145833,115.0,2.0,11.5,19,80,3,90,1,15,4,443,0.020833,1.0,0.0,0.0,1.0,-3.770833,1,-13,-3.0,0,1.0,1.0,0.104167,2.645833,14,3,23,39.000000,39.0,39.0,39.000000,154.0,8855.0,1782.229167,0.025974,1.493506,0.300595,438,3.968497
201914,2017-08-01,C_ID_7666735b3d,4,3,0,184,3,736,552,0,0.021739,0.016304,0.000000,7,0,2.333333,4,0,2.081666,1.066951,90,22175.0,

In [32]:
# pd.set_option('display.max_rows', None)
train.isnull().sum()

first_active_month               0
card_id                          0
feature_1                        0
feature_2                        0
feature_3                        0
                                ..
total_amount_month_ratio_min     0
total_amount_month_ratio_max     0
total_amount_month_ratio_mean    0
total_duration                   0
total_amount_change_ratio        0
Length: 63, dtype: int64

In [33]:
# for df in [train, test]:
#     df['hist_count_No'].fillna(0, inplace=True)
#     df['hist_ratio_No'].fillna(0, inplace=True)
#     df['new_amount_total'].fillna(0, inplace=True)
#     df['new_amount_min'].fillna(0, inplace=True)
#     df['new_amount_max'].fillna(0, inplace=True)
#     df['new_amount_mean'].fillna(0, inplace=True)
#     df['new_amount_median'].fillna(0, inplace=True)
#     df['new_merchant_category_nu'].fillna(0, inplace=True)
#     df['new_city_nu'].fillna(0, inplace=True)
#     df['new_state_nu'].fillna(0, inplace=True)
#     df['new_installments_sum'].fillna(0, inplace=True)
#     df['new_installments_min'].fillna(0, inplace=True)
#     df['new_installments_max'].fillna(0, inplace=True)
#     df['new_installments_mean'].fillna(0, inplace=True)
#     df['new_installments_median'].fillna(0, inplace=True)
#     df['new_price_sum'].fillna(0, inplace=True)
#     df['new_price_min'].fillna(0, inplace=True)
#     df['new_price_max'].fillna(0, inplace=True)
#     df['new_price_mean'].fillna(0, inplace=True)
    #df.fillna(0, inplace=True)

In [34]:
for df in [train, test]:
    df['ratio_amount_total'] = df['new_amount_total'] / df['hist_amount_total']
    df['ratio_amount_mean'] = df['new_amount_mean'] / df['hist_amount_mean']
    df['ratio_merchant_category_nu'] = df['new_merchant_category_nu'] / df['hist_merchant_category_nu']
    #df['ratio_installments_mean'] = df['new_installments_mean'] / df['hist_installments_mean']
    #df['ratio_installments_sum'] = df['new_installments_sum'] / df['hist_installments_sum']

KeyError: 'new_amount_total'

In [ ]:
train

In [ ]:
train.isnull().sum()

In [35]:
path_train_prepro = '../Kaggle-data/processed/oshikiri-kawai-train_processed.csv'
path_test_preprp = '../Kaggle-data/processed/oshikiri-kawai-test_processed.csv'

train.to_csv(path_train_prepro, index=False)
test.to_csv(path_test_preprp, index=False)


In [36]:
train

,first_active_month,card_id,feature_1,feature_2,feature_3,elapsed_time,quarter,days_feature1,days_feature2,days_feature3,days_feature1_ratio,days_feature2_ratio,days_feature3_ratio,feature_sum,feature_product,feature_mean,feature_max,feature_min,feature_var,target_raw,total_frequency,total_amount_total,total_amount_mean,total_amount_max,total_amount_min,total_amount_median,total_merchant_category_nu,total_merchant_category_mode,total_city_nu,total_city_mode,total_state_nu,total_state_mode,total_date_min,total_date_max,total_installments_mean,total_installments_max,total_installments_min,total_installments_median,total_installments_sum,total_month_lag_mean,total_month_lag_max,total_month_lag_min,total_month_lag_median,total_category_1_mod,total_category_2_mod,total_category_3_mod,total_weekend_mean,total_weekday_mean,total_day_nu,total_day_min,total_day_max,total_price_sum,total_price_min,total_price_max,total_price_mean,total_duration_min,total_duration_max,total_duration_mean,total_amount_month_ratio_min,total_amount_month_ratio_max,total_amount_month_ratio_mean,total_duration,total_amount_change_ratio
0,2017-06-01,C_ID_92a2005557,5,2,1,245,2,1225,490,245,0.020408,0.008163,0.004082,8,10,2.666667,5,1,2.081666,0.566331,283,21403.0,75.628975,2000.0,5.0,34.0,46,560,9,69,3,9,177,483,0.014134,1.0,0.0,0.0,4.0,-3.473498,2,-8,-3.0,0,1.0,1.0,0.339223,3.204947,31,1,31,202.000000,6.0,87.0,50.500000,385.0,154000.0,5818.448763,0.064935,25.974026,0.983046,305,25.444891
1,2017-01-01,C_ID_3d0044924f,4,1,0,396,1,1584,396,0,0.010101,0.002525,0.000000,5,0,1.666667,4,0,2.081666,1.313042,356,34299.0,96.345506,3578.0,3.0,25.0,58,307,9,69,3,9,5,453,1.556497,10.0,1.0,1.0,551.0,-4.921348,2,-12,-5.0,0,1.0,2.0,0.370787,3.331461,31,1,31,16083.738095,3.0,1789.0,45.434288,234.0,279084.0,7509.575843,0.038462,45.871795,1.236099,447,36.137176
2,2016-08-01,C_ID_d639edf6cd,2,2,0,549,3,1098,1098,0,0.003643,0.003643,0.000000,4,0,1.333333,2,0,1.154701,1.611111,44,1996.0,45.363636,400.0,11.0,31.5,9,705,5,143,2,5,10,482,0.000000,0.0,0.0,0.0,0.0,-8.363636,2,-13,-10.0,0,5.0,1.0,0.272727,3.340909,19,2,30,0.000000,NaN,NaN,NaN,847.0,30800.0,3489.318182,0.142857,5.194805,0.589767,472,7.817635
3,2017-09-01,C_ID_186d6a6901,4,3,0,153,3,612,459,0,0.026144,0.019608,0.000000,7,0,2.333333,4,0,2.081666,1.103813,84,5724.0,68.142857,1459.0,4.0,26.5,28,278,7,17,5,22,268,472,1.135802,3.0,1.0,1.0,92.0,-2.452381,2,-5,-3.5,0,4.0,2.0,0.166667,2.833333,26,2,31,4310.166667,4.0,1459.0,53.211934,308.0,112343.0,5244.071429,0.051948,18.948052,0.885473,203,20.410901
4,2017-11-01,C_ID_cdbd2c0db2,1,3,0,92,4,92,276,0,0.010870,0.032609,0.000000,4,0,1.333333,3,0,1.527525,0.895181,169,38344.0,226.887574,5284.0,0.0,49.0,37,278,7,17,7,22,314,482,1.297619,12.0,1.0,1.0,218.0,-0.680473,2,-3,-1.0,0,4.0,2.0,0.319527,3.248521,31,1,31,16933.333333,0.0,797.0,100.793651,0.0,401584.0,17340.721893,0.000000,69.526316,2.968742,167,22.289067
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201912,2017-09-01,C_ID_963962de2c,3,2,1,153,3,459,306,153,0.019608,0.013072,0.006536,6,6,2.000000,3,1,1.000000,0.149600,47,1189.0,25.297872,116.0,3.0,12.0,11,705,5,25,2,7,249,434,0.021277,1.0,0.0,0.0,1.0,-2.000000,1,-5,-2.0,0,3.0,1.0,0.361702,3.659574,22,1,29,45.000000,45.0,45.0,45.000000,228.0,8932.0,1947.531915,0.038961,1.506494,0.328613,184,3.585366
201913,2015-10-01,C_ID_1314773c0b,3,1,1,854,4,2562,854,854,0.003513,0.001171,0.001171,5,3,1.666667,3,1,1.154701,1.242217,48,1111.0,23.145833,115.0,2.0,11.5,19,80,3,90,1,15,4,443,0.020833,1.0,0.0,0.0,1.0,-3.770833,1,-13,-3.0,0,1.0,1.0,0.104167,2.645833,14,3,23,39.000000,39.0,39.0,39.000000,154.0,8855.0,1782.229167,0.025974,1.493506,0.300595,438,3.968497
201914,2017-08-01,C_ID_7666735b3d,4,3,0,184,3,736,552,0,0.021739,0.016304,0.000000,7,0,2.333333,4,0,2.081666,1.066951,90,22175.0,